In [28]:
import pandas as pd
import re

In [52]:
fairy_tales = pd.read_csv("../data/fairy_tales.csv")

chunks = []
sentences_per_chunk = 3
overlap_sentences = 1

for title, text in zip(fairy_tales["title"], fairy_tales["text"]):
    sentences = [s.strip().lower() for s in re.split(r"(?<=[.!?])\s+", text) if s.strip()]

    i = 0
    while i < len(sentences):
        chunk = " ".join(sentences[i:i + sentences_per_chunk])
        chunks.append(f"[{title.lower()}] {chunk}")
        i += sentences_per_chunk - overlap_sentences

print(f"chunks: {len(chunks)}")

chunks: 454


In [53]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [54]:
embed_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3")
embeddings = embed_model(tf.constant(chunks))

print(f"shape: {embeddings.shape}")

shape: (454, 512)


#### USE-QA

In [55]:
def answer_question(question, chunks, embeddings):
    question_embedding = embed_model([question])

    question_norm = tf.nn.l2_normalize(question_embedding, axis=1)
    chunks_norm = tf.nn.l2_normalize(embeddings, axis=1)

    similarities = tf.matmul(question_norm, chunks_norm, transpose_b=True)
    most_similar_idx = tf.argmax(similarities[0]).numpy()
    
    return chunks[most_similar_idx], similarities[0][most_similar_idx].numpy()

In [56]:
question = "Мысық нені төгеді?"
chunk, similarity = answer_question(question, chunks, embeddings)

print(f"Relevance: {similarity:.4f}")
print(f"Chunk: {chunk}")

Relevance: 0.6541
Chunk: [кедейдің үш ұлы] неге жылайсың? — дейді бұлар.— ой, қарақтарым-ай, мен жыламай, кім жыласын? бүгін жалғыз қызымнан айрылып отырмын, — дейді,— е, неге айрылдыңыз?— қарақтарым-ау, не қып дерің бар ма?


In [57]:
question = "Дүкенші мысықтан не сұрайды?"
chunk, similarity = answer_question(question, chunks, embeddings)

print(f"Relevance: {similarity:.4f}")
print(f"Chunk: {chunk}")

Relevance: 0.7873
Chunk: [кедейдің үш ұлы] неге жылайсың? — дейді бұлар.— ой, қарақтарым-ай, мен жыламай, кім жыласын? бүгін жалғыз қызымнан айрылып отырмын, — дейді,— е, неге айрылдыңыз?— қарақтарым-ау, не қып дерің бар ма?


#### Top-3

In [58]:
def answer_question(question, chunks, embeddings):
    question_embedding = embed_model([question])

    question_norm = tf.nn.l2_normalize(question_embedding, axis=1)
    chunks_norm = tf.nn.l2_normalize(embeddings, axis=1)

    similarities = tf.matmul(question_norm, chunks_norm, transpose_b=True)[0].numpy()

    most_similar_idx = similarities.argsort()[-3:][::-1]

    return [(chunks[i], similarities[i]) for i in most_similar_idx]

In [59]:
question = "Мысық нені төгеді?"
results = answer_question(question, chunks, embeddings)

for chunk, similarity in results:
    print(f"Relevance: {similarity:.4f} \nChunk: {chunk}\n")

Relevance: 0.6541 
Chunk: [кедейдің үш ұлы] неге жылайсың? — дейді бұлар.— ой, қарақтарым-ай, мен жыламай, кім жыласын? бүгін жалғыз қызымнан айрылып отырмын, — дейді,— е, неге айрылдыңыз?— қарақтарым-ау, не қып дерің бар ма?

Relevance: 0.6498 
Chunk: [аяз би] хан не дерін білмей, халыққа қарап:— апырым-ау, мына жаман не дейді! халқым, мен хан баласымын деп жүр едім, шақырыңдар анамды! — дейді.— ол о дүниеге біреудің қанын жүктеп барып, тәңірі алдында қара бет болмасын, не де болса шынын айтсын.

Relevance: 0.6440 
Chunk: [кедейдің үш ұлы] хан жылап көрісіп:— е, неғып босадың? — дейді.— бір жігіт келіп айдаһарды өлтіріп, мені тірі алып қалды, — дейді.— ол жігіт қайда?— қашып кетті.— қайда қашып кетті? неге айрылдың?



In [60]:
question = "Дүкенші мысықтан не сұрайды?"
results = answer_question(question, chunks, embeddings)

for chunk, similarity in results:
    print(f"Relevance: {similarity:.4f} \nChunk: {chunk}\n")

Relevance: 0.7873 
Chunk: [кедейдің үш ұлы] неге жылайсың? — дейді бұлар.— ой, қарақтарым-ай, мен жыламай, кім жыласын? бүгін жалғыз қызымнан айрылып отырмын, — дейді,— е, неге айрылдыңыз?— қарақтарым-ау, не қып дерің бар ма?

Relevance: 0.7362 
Chunk: [аяз би] хан не дерін білмей, халыққа қарап:— апырым-ау, мына жаман не дейді! халқым, мен хан баласымын деп жүр едім, шақырыңдар анамды! — дейді.— ол о дүниеге біреудің қанын жүктеп барып, тәңірі алдында қара бет болмасын, не де болса шынын айтсын.

Relevance: 0.7347 
Chunk: [кедейдің үш ұлы] хан жылап көрісіп:— е, неғып босадың? — дейді.— бір жігіт келіп айдаһарды өлтіріп, мені тірі алып қалды, — дейді.— ол жігіт қайда?— қашып кетті.— қайда қашып кетті? неге айрылдың?



#### TF-IDF

In [61]:
def answer_question(question, chunks, embeddings):
    question_embedding = embed_model([question])

    question_norm = tf.nn.l2_normalize(question_embedding, axis=1)
    chunks_norm = tf.nn.l2_normalize(embeddings, axis=1)

    similarities = tf.matmul(question_norm, chunks_norm, transpose_b=True)[0].numpy()

    most_similar_idx = similarities.argsort()[-3:][::-1]

    return most_similar_idx, similarities[most_similar_idx]

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [63]:
question = "Мысық нені төгеді?"
relevant_chunks, similarities = answer_question(question, chunks, embeddings)

vectorizer = TfidfVectorizer()
texts = [question] + [chunks[i] for i in relevant_chunks]
tfidf_matrix = vectorizer.fit_transform(texts)

tfidf_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

combined = 0.6 * similarities + 0.4 * tfidf_scores
most_relevant_chunk = relevant_chunks[combined.argmax()]

print(chunks[most_relevant_chunk])

[кедейдің үш ұлы] неге жылайсың? — дейді бұлар.— ой, қарақтарым-ай, мен жыламай, кім жыласын? бүгін жалғыз қызымнан айрылып отырмын, — дейді,— е, неге айрылдыңыз?— қарақтарым-ау, не қып дерің бар ма?


In [64]:
question = "Дүкенші мысықтан не сұрайды?"
relevant_chunks, similarities = answer_question(question, chunks, embeddings)

vectorizer = TfidfVectorizer()
texts = [question] + [chunks[i] for i in relevant_chunks]
tfidf_matrix = vectorizer.fit_transform(texts)

tfidf_scores = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]

combined = 0.6 * similarities + 0.4 * tfidf_scores
most_relevant_chunk = relevant_chunks[combined.argmax()]

print(chunks[most_relevant_chunk])

[кедейдің үш ұлы] неге жылайсың? — дейді бұлар.— ой, қарақтарым-ай, мен жыламай, кім жыласын? бүгін жалғыз қызымнан айрылып отырмын, — дейді,— е, неге айрылдыңыз?— қарақтарым-ау, не қып дерің бар ма?
